# Install dep

In [ ]:
!pip install --quiet transformers

# Import dep

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch.nn import functional as F

# Init and load models/tokenizers

In [ ]:
# Load GPT-2 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# Logic

In [23]:
# Tokenize input phrase
phrase = f'I sleep in a bed that is poorly iced, it\'s okay, it goes away. I\'ll make tea for you'
inputs = tokenizer.encode(phrase, return_tensors='pt')
print(inputs)

# Get logits from last layer
last_layer_logits = model(inputs).logits[:, -1, :]

# Keep top 30 logits at max; stop if cumulative probability >= 1.0.
top_logits = top_k_top_p_filtering(last_layer_logits, top_k=30, top_p=1.0)

# Softmax the logits into probabilities
probabilities = F.softmax(top_logits, dim=-1)

# Generate next token
generated_next_token = torch.multinomial(probabilities, num_samples=1)
generated = torch.cat([inputs, generated_next_token], dim=-1)

# Get result
result_string = tokenizer.decode(generated.tolist()[0])

# Print string
print(result_string)

tensor([[   40,  3993,   287,   257,  3996,   326,   318, 13455,   220,  3711,
            11,   340,   338,  8788,    11,   340,  2925,  1497,    13,   314,
          1183,   787,  8887,   329,   345]])
I sleep in a bed that is poorly iced, it's okay, it goes away. I'll make tea for you,
